In [ ]:
# default_exp transactions
# hide
_FNAME='transactions'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [ ]:
#export
import time
import os
import json
from functools import partial

from pathlib import Path
import covalent_api.constants as constants
import covalent_api as cov
from covalent_api.class_a import ClassA, constants

from sovrynkg.contracts import whatis, BY_NAME

RSK_CHAIN_ID = '30' #rsk
protocol = BY_NAME['sovrynProtocol']

class MustProvideCovalentAPIKey(Exception):
    pass

def classa_factory():
    constants.AVAILABLE_CHAIN_IDS['RSK Mainnet'] = RSK_CHAIN_ID
    api_key = os.environ.get('COVALENT_API_KEY', None)
    if not api_key:
        raise MustProvideCovalentAPIKey("Must set environment variable COVALENT_API_KEY")
    sess = cov.Session(api_key=api_key, timeout=15)
    sess._request.auth = (sess._api_key, '')
    return ClassA(sess)    

CLASSA = classa_factory()

In [ ]:
#export
def get_num_results(result):
    return len(result['data']['items'])

def get_block_height(result, index=-1):
    return result['data']['items'][index]['block_height']

def save_result_to_file(result, page_number, directory):
    first_block = get_block_height(result, index=0)
    last_block = get_block_height(result, index=-1)
    fname = 'result.{fb}__{lb}'.format(fb=first_block, lb=last_block)

    Path(directory).mkdir(exist_ok=True, parents=True)
    with open(os.path.join(directory, fname), 'w') as f:
        json.dump(result, f, indent=2)

def max_pages_exceeded(pages, max_pages):
    if max_pages is None:
        return False
    return pages>max_pages
        
def paginate(partial_func, page_size=10000, page_number=0, max_pages=10000, save_in_dir=None, in_memory=True):
    '''
    Create a partial function that accepts page_number and page_size kwargs
    '''
    num_results = None
    keep_going = True 
    results = []
    files_saved = 0
    pages_retrieved = 0
    
    while keep_going and not max_pages_exceeded(pages_retrieved, max_pages):
        result = partial_func(page_number=page_number, page_size=page_size)
        pages_retrieved += 1
        
        if result['error']:
            keep_going = False
            return result
        else:
            num_results = get_num_results(result)
        
        if num_results == 0:
            keep_going = False
        else: #deal with results
            print("Got {num_results} results for page {pn}".format(num_results=num_results, pn=page_number))
            block_height = result['data']['items'][-1]['block_height']
            
            print("Block height {}".format(block_height))
            if in_memory:
                results.append(result)

            if save_in_dir:
                save_result_to_file(result=result, page_number=page_number, directory=save_in_dir)
                files_saved += 1                    
                    
            page_number += 1
            time.sleep(0.1)
                    
    if in_memory:
        return results
    else:
        return page_number
    
def get_transactions(address, **kwargs):
    partial_func = partial(CLASSA.get_transactions,
                           chain_id=RSK_CHAIN_ID, address=address)
    return paginate(partial_func, **kwargs)

To get transactions
```python
pages = get_transactions(address=protocol.address, page_size=5, max_pages=1)
```
or for a bigger harvest
```python
get_transactions(address=protocol.address, page_size=500, max_pages=None, save_in_dir='protocol_transactions', in_memory=False)
```

In [ ]:
#import yaml
#with open('testtransactions.yaml') as f:
#    pages = yaml.safe_load(f)
    
pages = [
    {'data': {'items':[

        {'block_height': 3502536,
 'block_signed_at': '2021-07-09T16:36:00Z',
 'from_address': '0xd01c33b8d585eb8617791449d847d31767a5723a',
 'from_address_label': None,
 'gas_offered': 2500000,
 'gas_price': 71680400,
 'gas_quote': 0.12745898634319577,
 'gas_quote_rate': 2162.3291015625,
 'gas_spent': 822334,
 'successful': True,
 'to_address': '0x5a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7',
 'to_address_label': None,
 'tx_hash': '0x5ef38a4e6944f6dc1511ae77f34e2dfbefd588b36c79a477d37b7e7b0aa43a66',
 'tx_offset': 0,
 'value': '0',
 'value_quote': 0.0,
 'log_events': [
     {'_raw_log_topics_bytes': None,
 'block_height': 3502536,
 'block_signed_at': '2021-07-09T16:36:00Z',
 'decoded': None,
 'log_offset': 26,
 'raw_log_data': '0x00000000000000000000000000000000000000000000000000001ec08ab86504',
 'raw_log_topics': ['0xc44aeefa68e8b9c1ad5f7be4b0dd194580f81f5c362862e72196503a320eb7a1',
  '0x000000000000000000000000542fda317318ebf1d3deaf76e0b632741a7e677d',
  '0x000000000000000000000000d01c33b8d585eb8617791449d847d31767a5723a'],
 'sender_address': '0x5a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7',
 'sender_address_label': None,
 'sender_contract_decimals': None,
 'sender_contract_ticker_symbol': None,
 'sender_logo_url': None,
 'sender_name': None,
 'tx_hash': '0x5ef38a4e6944f6dc1511ae77f34e2dfbefd588b36c79a477d37b7e7b0aa43a66',
 'tx_offset': 0},
{'_raw_log_topics_bytes': None,
 'block_height': 3502536,
 'block_signed_at': '2021-07-09T16:36:00Z',
 'decoded': {'name': 'Withdrawal',
  'params': [{'decoded': True,
    'indexed': True,
    'name': 'src',
    'type': 'address',
    'value': '0x5a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7'},
   {'decoded': True,
    'indexed': False,
    'name': 'wad',
    'type': 'uint256',
    'value': '33812309894404'}],
  'signature': 'Withdrawal(indexed address src, uint256 wad)'},
 'log_offset': 25,
 'raw_log_data': '0x00000000000000000000000000000000000000000000000000001ec08ab86504',
 'raw_log_topics': ['0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65',
  '0x0000000000000000000000005a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7'],
 'sender_address': '0x542fda317318ebf1d3deaf76e0b632741a7e677d',
 'sender_address_label': None,
 'sender_contract_decimals': 18,
 'sender_contract_ticker_symbol': 'WRBTC',
 'sender_logo_url': '',
 'sender_name': 'Wrapped BTC',
 'tx_hash': '0x5ef38a4e6944f6dc1511ae77f34e2dfbefd588b36c79a477d37b7e7b0aa43a66',
 'tx_offset': 0}    
 ]}
    ]}}
]

Anatomy of the results returned from pagination.


List. Each entry is one page.

In [ ]:
firstpage = pages[0]
firstpage.keys()

dict_keys(['data'])

In [ ]:
data = firstpage['data']
data.keys()

dict_keys(['items'])

In [ ]:
items = data['items']
items[0]

{'block_height': 3502536,
 'block_signed_at': '2021-07-09T16:36:00Z',
 'from_address': '0xd01c33b8d585eb8617791449d847d31767a5723a',
 'from_address_label': None,
 'gas_offered': 2500000,
 'gas_price': 71680400,
 'gas_quote': 0.12745898634319577,
 'gas_quote_rate': 2162.3291015625,
 'gas_spent': 822334,
 'successful': True,
 'to_address': '0x5a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7',
 'to_address_label': None,
 'tx_hash': '0x5ef38a4e6944f6dc1511ae77f34e2dfbefd588b36c79a477d37b7e7b0aa43a66',
 'tx_offset': 0,
 'value': '0',
 'value_quote': 0.0,
 'log_events': [{'_raw_log_topics_bytes': None,
   'block_height': 3502536,
   'block_signed_at': '2021-07-09T16:36:00Z',
   'decoded': None,
   'log_offset': 26,
   'raw_log_data': '0x00000000000000000000000000000000000000000000000000001ec08ab86504',
   'raw_log_topics': ['0xc44aeefa68e8b9c1ad5f7be4b0dd194580f81f5c362862e72196503a320eb7a1',
    '0x000000000000000000000000542fda317318ebf1d3deaf76e0b632741a7e677d',
    '0x000000000000000000000000d01

In [ ]:
import sha3
def keccak(inputstr):
    encoded = inputstr.encode()
    keccak = sha3.keccak_256()
    keccak.update(encoded)
    return keccak.hexdigest()

#https://codeburst.io/deep-dive-into-ethereum-logs-a8d2047c7371
TESTCASE.assertEqual('ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
                     keccak('Transfer(address,address,uint256)'))

Withdrawl abi from [Wrapped BTC](https://explorer.rsk.co/address/0x542fda317318ebf1d3deaf76e0b632741a7e677d?__ctab=Code)
```json
  {
    "anonymous": false,
    "inputs": [
      {
        "indexed": true,
        "internalType": "address",
        "name": "src",
        "type": "address"
      },
      {
        "indexed": false,
        "internalType": "uint256",
        "name": "wad",
        "type": "uint256"
      }
    ],
    "name": "Withdrawal",
    "type": "event"
  }
```

## Re-create this known topic

In [ ]:
keccak('Withdrawal(address,uint256)')

'7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65'

In [ ]:
#logs[1]['raw_log_topics'][0]
'0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65'

'0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65'

## Re-create a topic from the SoV source code

In [ ]:
delegated_manager_set='DelegatedManagerSet(bytes32,address,address,bool)'
keccak(delegated_manager_set)

'0eef4f90457a741c97d76fcf13fa231fefdcc7649bdb3cb49157c37111c98433'

```js
/// topic0: 0x0eef4f90457a741c97d76fcf13fa231fefdcc7649bdb3cb49157c37111c98433
event DelegatedManagerSet(bytes32 indexed loanId, address indexed delegator, address indexed delegated, bool isActive);
```

## Calculate unknown topics from the sov source code

In [ ]:
loanswap = 'LoanSwap(bytes32,address,address,address,uint256,uint256)'
keccak(loanswap)

'b4eb3c9b62efcce7021cba5fd9cd0c44df91c2272806ccc5e57df7c912e8d716'

In [ ]:
notebook2script(_nbpath)

Converted transactions.ipynb.
